<a href="https://colab.research.google.com/github/Aleena-Mishra-10/Deep-Learning/blob/master/Flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense , Dropout ,  Activation , Flatten
from keras.layers import Conv2D , MaxPooling2D , ZeroPadding2D
from keras.layers.normalization  import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint , EarlyStopping , ReduceLROnPlateau
from keras.models import Model

In [0]:
train_dir='/content/drive/My Drive/17_flowers/train'
validation_dir='/content/drive/My Drive/17_flowers/validation'

In [0]:
num_classes=17
img_cols=224
img_rows=224
train_batchsize=16
val_batchsize=16

In [0]:
train_datagen=ImageDataGenerator(
    rescale=1./255  ,
    rotation_range= 20 ,
    width_shift_range=0.2 ,
    height_shift_range=0.2 ,
    horizontal_flip=True ,
    fill_mode='nearest'
)

valid_datagen=ImageDataGenerator(
    rescale=1./255
)

train_generator=train_datagen.flow_from_directory(
          train_dir,
          target_size=(img_rows,img_cols),
          batch_size=train_batchsize,
          class_mode='categorical'    
)

valid_generator=valid_datagen.flow_from_directory(
          validation_dir,
          target_size=(img_rows,img_cols),
          batch_size=val_batchsize,
          class_mode='categorical',
          shuffle=False
)

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


In [0]:
vgg16=VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=(img_rows,img_cols,3)
)

In [0]:
for (i,layer) in enumerate(vgg16.layers):
  print(str(i)+" "+layer.__class__.__name__ , layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [0]:
for layer in vgg16.layers:
  layer.trainable=False


for (i,layer) in enumerate(vgg16.layers):
  print(str(i)+" ",layer.__class__.__name__,layer.trainable)

0  InputLayer False
1  Conv2D False
2  Conv2D False
3  MaxPooling2D False
4  Conv2D False
5  Conv2D False
6  MaxPooling2D False
7  Conv2D False
8  Conv2D False
9  Conv2D False
10  MaxPooling2D False
11  Conv2D False
12  Conv2D False
13  Conv2D False
14  MaxPooling2D False
15  Conv2D False
16  Conv2D False
17  Conv2D False
18  MaxPooling2D False


In [0]:
def addTopModel(vgg16,num_classes,D=256):
  top_model=vgg16.output
  top_model=Flatten(name='flatten')(top_model)
  top_model=Dense(D , activation='relu')(top_model)
  top_model=Dropout(0.3)(top_model)
  top_model=Dense(num_classes,activation='softmax')(top_model)
  return top_model

In [0]:
FC_Head=addTopModel(vgg16,num_classes)
model=Model(inputs=vgg16.input,output=FC_Head)
print(model.summary())



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [0]:
checkpoint = ModelCheckpoint(
    "/This PC/DownloadTrained Models/flowers_vgg.h5",
    monitor="val_loss",
    mode="min",
    save_best_only=True,
    verbose=1
)

earlystop=EarlyStopping(
      monitor='val_loss',
      min_delta=0,
      patience=3,
      verbose=1,
      restore_best_weights=True
)

callbacks=[checkpoint,earlystop]

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(lr=0.001),
    metrics=['accuracy']
)

In [0]:
nb_train_sample=1190
nb_validation_sample=170
epochs=3
batch_size=16


In [0]:
history=model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_sample//batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=valid_generator,
    validation_steps=nb_validation_sample//batch_size
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/3
37/74 [==============>...............] - ETA: 5:13 - loss: 12.8859 - acc: 0.0811

KeyboardInterrupt: ignored